In [ ]:
! pip install qiskit

In [ ]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile
from qiskit.circuit.library import ZGate, GroverOperator, MCMT
from qiskit.providers.basic_provider import BasicProvider
import math


In [ ]:
def oracle(marked, k):
    """Builds an oracle using Grover's algorithm to mark states
        of odd numbers
    Parameters:
        marked (list): Marked States
        k (int): Number of qubits to use

    Returns:
        QuantumCircuit: A quantum circuit
    """
    circuit = QuantumCircuit(k)
    lst = []

    for target in marked:
        flip_target = target[::-1]
        if flip_target[1:] in lst:
            continue
        indices = [idx for idx, val in enumerate(flip_target) if val == "0" and idx != 0]

        if not indices:
          indices = [0, k-1]
        # Add a multi-controlled Z-gate with pre- and post-applied X-gates (open-controls)
        circuit.x(indices)
        circuit.compose(MCMT(ZGate(), k - 1, 1), inplace=True)
        circuit.x(indices)
        lst.append(flip_target[1:])

    return circuit

def odd_to_even(n, lst):
    """
    Quantum algorithm that when given numbers of range [1,n]
    and are odd convert them into even numbers

    Parameters:
        n (int): threshold number
        lst (list): List of numbers

    Returns:
        ans_list: A list where all numbers are even and in range [1, n]
    """
    k = math.ceil(math.log2(n))

    bin_marked = [bin(num)[2:].zfill(k) for num in lst]

    prep_oracle = oracle(bin_marked, k)
    # Count the number of multi-control z gate in the oracle.
    num_marked = dict(prep_oracle.count_ops())["CCX" if k==3 else f"c{k-1}z"]
    grover = GroverOperator(prep_oracle)

    # Determine the optimal number of iterations
    iterations = max(1, math.floor(math.pi / (4 * math.asin(math.sqrt(5 / 2**grover.num_qubits)))))
    circuit = QuantumCircuit(k)

    # Create an even superposition of all basis states
    circuit.h(range(grover.num_qubits))
    # Apply Grover operator the optimal number of times
    circuit.compose(grover.power(iterations), inplace=True)
    circuit.measure_all()

    backend = BasicProvider().get_backend('basic_simulator')
    transpiled_circuit = transpile(circuit, backend)
    counts = backend.run(transpiled_circuit, shots=10000).result().get_counts()

    odd_numbers = sorted(counts, key=counts.get, reverse=True)[:num_marked]

    odd_numbers = [int(i, 2) for i in odd_numbers]

    # Delete 0 from the list since we have to remain within [1, n)
    if 0 in odd_numbers:
        odd_numbers.remove(0)

    ans_list = []

    for i in lst:
        if i not in odd_numbers and i < n:
            ans_list.append(i)
        elif i+1 not in odd_numbers and i < n:
            ans_list.append(i+1)
        elif i-1 not in odd_numbers and i < n:
            ans_list.append(i-1)
        else:
            ans_list.append(n-1)

    return ans_list

B = odd_to_even (n = 31,lst= [1,2,2,4,5,6,7,11,17,21,22,23] )
print(B)

[2, 2, 2, 4, 6, 6, 8, 12, 18, 22, 22, 24]
